In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
#!pip install cuml-cu11 --extra-index-url=https://pypi.nvidia.com
#import cuml
#from cuml.ensemble import RandomForestClassifier


Mounted at /content/gdrive/


In [ ]:
df = pd.read_csv(r'/content/gdrive/MyDrive/patent project/clean_data_with_enrich.csv',dtype={'filing_year':'str'})

<ipython-input-2-7ba450f1f037>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'/content/gdrive/MyDrive/patent project/clean_data_with_enrich.csv',dtype={'filing_year':'str'})


In [ ]:
df = df[(df['one_if_small']==1) & (df['Patent_Target']!='pending')]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.inspection import PartialDependenceDisplay,partial_dependence
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import GridSearchCV


In [3]:
import pickle
pickle_file = open(r'/content/drive/MyDrive/patent project/patnet_topic.pickle', 'rb')
topic_dict = pickle.load(pickle_file)

In [4]:
len(topic_dict)

1150

In [ ]:
for word in topic_list:
    df[word] = df['clean_abstract'].apply(lambda x: 1 if word in x else 0)

<ipython-input-31-4d6d7ac05c24>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[word] = df['clean_abstract'].apply(lambda x: 1 if word in x else 0)
<ipython-input-31-4d6d7ac05c24>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[word] = df['clean_abstract'].apply(lambda x: 1 if word in x else 0)
<ipython-input-31-4d6d7ac05c24>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

In [ ]:
rej_df = df.drop(['application_number','one_if_male','one_if_patented','one_if_abandoned','one_if_pending','abstract','clean_abstract','Gender_Target','Parent_Catgeory'],axis=1)

In [ ]:
rej_df = pd.get_dummies(rej_df,columns=['uspc_class'],drop_first=True)

In [ ]:
rej_df['Patent_Target'] = rej_df['Patent_Target'].apply(lambda x: 1 if x=='abandoned' else 0)

In [ ]:
rej_df.to_csv(r'/content/gdrive/MyDrive/patent project/df_for_rf.csv',index=False)

In [2]:
rej_df = pd.read_csv(r'/content/gdrive/MyDrive/patent project/df_for_rf.csv')

In [10]:
rej_df.columns.to_list()

['one_if_female',
 'filing_year',
 'one_if_small',
 'Biotechnology',
 'Chem_Material_Engr',
 'Comp_Archit',
 'Comp_Net',
 'Comms',
 'Semis',
 'Trans',
 'Mech_Engr',
 'kinkade',
 'word_count',
 'char_count',
 'median_word_length',
 'avg_word_length',
 'skew_word_length',
 'characters_per_word',
 'syll_per_word',
 'words_per_sentence',
 'sentences_per_paragraph',
 'type_token_ratio',
 'syllables',
 'sentences',
 'long_words',
 'complex_words',
 'Patent_Target',
 'inlet',
 'formulation',
 'equipment',
 'mounting',
 'assembling',
 'dispenser',
 'specification',
 'compartment',
 'wheeled',
 'concrete',
 'dispenses',
 'sensor',
 'panel',
 'attachment',
 'procedure',
 'constructing',
 'enclosure',
 'assembly',
 'vehicle',
 'generator',
 'accessory',
 'functionality',
 'database',
 'implementing',
 'outlet',
 'rotary',
 'shaft',
 'formulated',
 'alternator',
 'wireless',
 'embodiment',
 'cultivar',
 'infant',
 'pivoting',
 'chassis',
 'prescription',
 'automotive',
 'methodology',
 'accelerome

In [ ]:
rej_df = rej_df[rej_df['Chem_Material_Engr']== 1]

In [ ]:
X = rej_df.drop('Patent_Target', axis=1)
y = rej_df['Patent_Target']


In [ ]:
rej_df['Patent_Target'].value_counts(normalize=True)

0    0.59526
1    0.40474
Name: Patent_Target, dtype: float64

In [ ]:
import tensorflow as tf


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

with tf.device('/device:GPU:0'):
  for i in range(50,500,50):
    for d in range(10,160,20):
      clf = cuml.ensemble.RandomForestClassifier(n_estimators=i,max_depth=d)
      clf.fit(X_train, y_train)
      y_pred = clf.predict(X_test)
      print("num estimators",i,'max depth',d)
      print(accuracy_score(y_test,y_pred))



num estimators 50 max depth 10
0.6282249941891996


/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


num estimators 50 max depth 30
0.6485627953823506


/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


num estimators 50 max depth 50
0.6547609824126444


...

num estimators 450 max depth 110
0.6586348493065778


/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


num estimators 450 max depth 130
0.6591384520027892


/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


num estimators 450 max depth 150
0.6590222359959712


In [ ]:
with tf.device('/device:GPU:0'):
  clf = cuml.ensemble.RandomForestClassifier(n_estimators=350,max_depth=110)
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print("num estimators",i,'max depth',d)
  print(accuracy_score(y_test,y_pred))


/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


num estimators 450 max depth 150
0.6604168280777872


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
clf = RandomForestClassifier(n_estimators=350,max_depth=110)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test,y_pred))


0.6623150228558147


In [ ]:
PartialDependenceDisplay.from_estimator(clf, X, ['type_token_ratio'])

KeyboardInterrupt: ignored

In [ ]:
print(accuracy_score(y_test,y_pred))


0.6959666544072576


In [ ]:
importances = clf.feature_importances_


In [ ]:
feature_importances_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})


In [ ]:
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)


In [ ]:
feature_importances_df.head(50)

,Feature,Importance
1570,uspc_class_705,0.041429
21,type_token_ratio,0.032039
18,syll_per_word,0.031615
17,characters_per_word,0.030398
15,avg_word_length,0.030110
16,skew_word_length,0.028099
11,kinkade,0.027593
13,char_count,0.027483
22,syllables,0.026634
12,word_count,0.026125


In [ ]:
rej_df['']

In [ ]:
y_test.shape

(25814,)

In [ ]:
y_pred.shape

(25814,)

In [ ]:
!pip install keybert
from keybert import KeyBERT

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.3 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23777 sha256=b16cd37c8e4aeefa66b5794d290ac85dbc698962061e57e8d98f3159615a3f46
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=5d00285464d286110849b87ec0c017248b021a715b85176b6a7da5f52debfca7


In [ ]:
absract_df = df[['clean_abstract']].head(20)

In [ ]:
absract_df['Topics'] = absract_df['clean_abstract'].apply(lambda x: KeyBERT().extract_keywords(x,
                                    keyphrase_ngram_range=(1, 1), top_n=2))

In [ ]:
df_topics = pd.read_csv(r'/content/gdrive/MyDrive/patent project/patented_df_topics.csv')

In [ ]:
KeyBERT().extract_keywords(df_topics.iloc[0,1],
                                    keyphrase_ngram_range=(1, 1), top_n=3)

[('soybean', 0.4643), ('embodiment', 0.3394), ('cultivar', 0.2819)]

In [ ]:
topic_dict = {}

for i in range(df_topics.shape[0]):
  topics = KeyBERT().extract_keywords(df_topics.iloc[i,1],
                                    keyphrase_ngram_range=(1, 1), top_n=3)
  for topic in topics:
    if topic[0] in topic_dict:
      topic_dict[topic[0]]+=1
    else:
      topic_dict[topic[0]]=1

In [ ]:
pd.set_option('display.max_colwidth', None)



In [ ]:
import pickle
fHandle = open(r'/content/gdrive/MyDrive/patent project/patnet_topic.pickle','wb')
pickle.dump(topic_dict, fHandle,protocol=pickle.HIGHEST_PROTOCOL)
fHandle.close()

In [ ]:
len(topic_dict)

1150

In [ ]:
topic_list = [x[0] for x in sorted(topic_dict.items(),key=lambda x: x[1],reverse=True)]

In [ ]:
len(topic_list)

1150

In [ ]:
text_df = df[['clean_abstract']]

In [ ]:
for word in topic_list:
    text_df[word] = text_df['clean_abstract'].apply(lambda x: 1 if word in x else 0)

<ipython-input-83-4d0881e2cbc7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df[word] = text_df['clean_abstract'].apply(lambda x: 1 if word in x else 0)
<ipython-input-83-4d0881e2cbc7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df[word] = text_df['clean_abstract'].apply(lambda x: 1 if word in x else 0)
<ipython-input-83-4d0881e2cbc7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [ ]:
text_df.head()

,clean_abstract,inlet,formulation,equipment,mounting,assembling,dispenser,specification,compartment,wheeled,...,basket,mat,flooring,incline,eyelash,lash,eyeliner,transgenically,transgenic,cardholder
2,present invention provides adjuvant composition improved stability increased potency provide enhanced h response present invention also provides method making composition administration improved adjuvant composition,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,disclosed invention relates receiver tube heating fluid molten salt oil used conjunction trough reflector concentric concentrators disclosed invention utilizes fused silica receiver tube assembly provide optical absorption way medium imbedded within thermal transfer fluid preferably comprising inorganic dye comprise pulverized thin film coating dissolved material specifically designed maximizing optical absorption alternatively chemistry transfer fluid modified increase optical absorption optically absorbing medium may comprise fine powder density preferably similar thermal transfer fluid fine graphite powder another preferred embodiment absorbing mean within heat transfer fluid comprise solid absorbing element disposed along central axis receiver tube interior,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,lamp assembly may include linear array reflecting surface arranged limit angular distribution direct light supplementing reflected light intensity direct light flat surface object illuminated reflecting surface may shaped cause distribution total illumination illuminated portion flat surface uniform linearly tapered another desired profile reflecting surface may part reflector may include mounting surface blind oblong mounting hole allowing rotational adjustment heat sink mounting hole one exit hole may end piece attached,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,one embodiment includes launch vessel defining elongate linear interior extending bottom portion exit opening embodiment includes ram slidably disposed launch vessel ram sealed vessel embodiment also includes one wedge coupled launch vessel along interior proximal exit opening wedge shape sized increasingly narrow cross section interior along exit vector extending bottom portion toward exit opening embodiment vessel house charge proximal bottom portion charge propel ram along exit vector one wedge sized stop ram inside interior,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,present invention relates bottle cap comprising insertion part configured fitted inner circumference slider blocking inner circumference slider sealing ring configured extended lateral side lower end insertion part sealing lower end slider extension pipe configured extended lower side sealing ring least one shrinkage inducing groove cut lengthwise extension pipe diameter extension pipe decreased outside pressing force hook protruding outer side extension pipe outward direction extension pipe engaged bottom surface locking protrusion inner circumferential side end portion inner cap,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
